<a href="https://colab.research.google.com/github/FrioKristoffPBillanes/E-comProject/blob/main/ecom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json

with open("ecomdata.json") as f:
  data = json.load(f)

print(data)

[{'order_id': 1, 'customer': 'Alice Brown', 'order_date': '2025-08-01', 'status': 'Delivered', 'payment_method': 'Credit Card', 'shipping_address': '123 Main St, Quezon City', 'items': [{'product': 'Headphones', 'category': 'Electronics', 'price': 1500, 'quantity': 1}, {'product': 'USB-C Cable', 'category': 'Accessories', 'price': 250, 'quantity': 2}], 'total_amount': 2000}, {'order_id': 2, 'customer': 'Bob White', 'order_date': '2025-08-03', 'status': 'Pending', 'payment_method': 'Cash on Delivery', 'shipping_address': '45 Mabini St, Davao City', 'items': [{'product': 'Smartphone', 'category': 'Electronics', 'price': 22000, 'quantity': 1}, {'product': 'Phone Case', 'category': 'Accessories', 'price': 500, 'quantity': 1}], 'total_amount': 22500}, {'order_id': 3, 'customer': 'Charlie Green', 'order_date': '2025-08-05', 'status': 'Delivered', 'payment_method': 'GCash', 'shipping_address': '78 Rizal Ave, Cebu City', 'items': [{'product': 'Book', 'category': 'Books', 'price': 450, 'quantit

In [2]:
!pip install firebase-admin

In [3]:
import firebase_admin
from firebase_admin import credentials, db

cred = credentials.Certificate("firebasekey.json")

firebase_admin.initialize_app(cred, {
  "databaseURL": "https://e-com-f57a7-default-rtdb.asia-southeast1.firebasedatabase.app/"
})
print("Firebase connected successfully!")

Firebase connected successfully!


In [4]:
with open("ecomdata.json", "r") as file:
  data = json.load(file)
print("JSON file loaded")
ref = db.reference("orders")
for order in data:
  ref.child(str(order["order_id"]).strip()).set(order)
print("Data uploaded successfully!")

JSON file loaded
Data uploaded successfully!


In [5]:
books_ref = db.reference("orders")
data = books_ref.get()
print(data)

[None, {'customer': 'Alice Brown', 'items': [{'category': 'Electronics', 'price': 1500, 'product': 'Headphones', 'quantity': 1}, {'category': 'Accessories', 'price': 250, 'product': 'USB-C Cable', 'quantity': 2}], 'order_date': '2025-08-01', 'order_id': 1, 'payment_method': 'Credit Card', 'shipping_address': '123 Main St, Quezon City', 'status': 'Delivered', 'total_amount': 2000}, {'customer': 'Bob White', 'items': [{'category': 'Electronics', 'price': 22000, 'product': 'Smartphone', 'quantity': 1}, {'category': 'Accessories', 'price': 500, 'product': 'Phone Case', 'quantity': 1}], 'order_date': '2025-08-03', 'order_id': 2, 'payment_method': 'Cash on Delivery', 'shipping_address': '45 Mabini St, Davao City', 'status': 'Pending', 'total_amount': 22500}, {'customer': 'Charlie Green', 'items': [{'category': 'Books', 'price': 450, 'product': 'Book', 'quantity': 1}, {'category': 'Stationery', 'price': 300, 'product': 'Notebook Set', 'quantity': 2}], 'order_date': '2025-08-05', 'order_id': 3

In [ ]:
filtered_data = [o for o in data if o is not None]
data = filtered_data

def crorders(data):
    name = input("Enter customer name: \n")
    results = []
    for order in filtered_data:
        if order["customer"].lower() == name.lower():
            results.append(order)
    return results

def print_order(order):
    print(f"\nOrder ID: {order['order_id']}")
    print(f"Customer: {order['customer']}")
    print(f"Order Date: {order['order_date']}")
    print(f"Status: {order['status']}")
    print(f"Payment Method: {order['payment_method']}")
    print(f"Address: {order['shipping_address']}")
    print("Items:")

    for item in order["items"]:
        print(f"  - {item['product']} (₱{item['price']} × {item['quantity']})")

    print(f"Total Amount: ₱{order['total_amount']}")
    print("-" * 40)

def sales_summary(data):
    orders = len(filtered_data)
    print(f"Total number of orders: {orders}")
    total_sales = 0
    for order in filtered_data:
        total_sales += sum(item['price'] * item['quantity'] for item in order['items'])
    print(f"Total sales amount: ₱{total_sales}")
    avg_value = total_sales / orders if orders > 0 else 0
    print(f"Average order value: ₱{avg_value}")

    menu()

def category_summary(data):
    category = input("Enter category to summarize (Electronics, Accessories, Books, Stationery, Sportswear): \n").lower()

    total_sales = 0
    total_quantity = 0
    found = False

    for order in filtered_data:
        for item in order['items']:
            if item['category'].lower() == category:
                found = True
                total_sales += item['price'] * item['quantity']
                total_quantity += item['quantity']

    if not found:
        print("Category doesn't exist.")
        menu()

    print(f"\nCategory: {category.capitalize()}")
    print(f"Total Sales: ₱{total_sales}")
    print(f"Total Quantity Sold: {total_quantity}")

    menu()

def pending_orders(data):
    print("\nPending Orders:")

    for order in filtered_data:
        if order['status'].lower() != "delivered":
            print_order(order)

    menu()

def best_product(data):
    sales = {}

    for order in filtered_data:
        for item in order['items']:
            product = item['product']
            qty = item['quantity']
            sales[product] = sales.get(product, 0) + qty

    sorted_products = sorted(sales.items(), key=lambda x: x[1], reverse=True)

    top_3 = sorted_products[:3]

    print("\nTop 3 Best-Selling Products:")
    for product, qty in top_3:
        print(f"• {product}: {qty} sold")

    menu()


def menu():
    print("\nE-Shop Analyzer")
    print("Welcome! What would you like to do today?")
    print("[1] Sales Summary")
    print("[2] Top Products")
    print("[3] Category Summary")
    print("[4] View Pending Orders")
    print("[5] Search Orders by Customer Name")
    print("[6] Exit Program")
    choice = int(input("Enter choice: \n"))
    if choice == 1:
        sales_summary(data)
    elif choice == 2:
        best_product(data)
    elif choice == 3:
        category_summary(data)
    elif choice == 4:
        pending_orders(data)
    elif choice == 5:
        orders = crorders(data)
        for o in orders:
            print_order(o)
        menu()
    elif choice == 6:
        print("Exiting program. Goodbye!")
        exit()
    else:
        print("Invalid choice. Please try again.")
        menu()

menu()


E-Shop Analyzer
Welcome! What would you like to do today?
[1] Sales Summary
[2] Top Products
[3] Category Summary
[4] View Pending Orders
[5] Search Orders by Customer Name
[6] Exit Program
Enter choice: 
1
Total number of orders: 4
Total sales amount: ₱29650
Average order value: ₱7412.5

E-Shop Analyzer
Welcome! What would you like to do today?
[1] Sales Summary
[2] Top Products
[3] Category Summary
[4] View Pending Orders
[5] Search Orders by Customer Name
[6] Exit Program
Enter choice: 
2

Top 3 Best-Selling Products:
• USB-C Cable: 2 sold
• Notebook Set: 2 sold
• Headphones: 1 sold

E-Shop Analyzer
Welcome! What would you like to do today?
[1] Sales Summary
[2] Top Products
[3] Category Summary
[4] View Pending Orders
[5] Search Orders by Customer Name
[6] Exit Program
